# Description

## Parameters setting
- `data_directory`: path of directory where the data is stored.
- `number_of_individuals_for_train`: the number of individual for how many individuals would be participated in training phase.
- `device`: Which processing unit would be used for PyTorch? 'cpu' or 'cuda'. If you have multiple gpu and specify HW, try 'cuda:0' or others.
- `epoch`: epoch number of training
- `batch_size`: batch_size
- `model_name`: a name that you want to call the model.
- `model_dir`: a directory that the trained model would be saved and loaded.
- `result_dir`: a directory that the figures and performance score would be saved.
- `lrFE,SV,REG`: learning rate of feature extractor, severity regressor, and regularizer network block.
- `lossREG`: ratio of loss for regularizer network block.
- `beta1,2`: parameters of ADAM optimizer.

In [ ]:
data_directory = 'data_for_paper/'
number_of_individuals_for_train = 32
device = "cuda"
epoch=150000
batch_size=64
model_name = 'regL'
model_dir='model'
result_dir = "result"
lrFE=0.0002
lrSV=0.0002
lrREG=0.0002
lossREG=0.002
beta1=0.5
beta2=0.999


: 

## Set-up data
DataManger instance would manage data processing, such as loadding, normalizing. It would take some memory to store the data.

In [ ]:
from lib.data_manager import DataManager
import numpy as np
DM = DataManager(path=data_directory)
DM.loads_data()
DM.normalize_data()
rndtr = np.random.randint(0, DM.data_input_train.shape[0], number_of_individuals_for_train)

## Train model
Regressor class is a network configure defined with PyTorch.

IMPORTANT! There are various scripts for regressor, (e.g. regressor_basic.py, regressor_regularize_L.py, ... so on) and they have difference for the network configuration. As you guess from the name of script, the difference on configuration could be existence of regularizer or multitasking.

In [ ]:
from lib.regressor_basic import Regressor

Reg=Regressor(epoch=epoch, batch_size=batch_size,
                model_dir="{}/{}".format(model_dir, model_name),
                lrFE=lrFE, lrSV=lrSV, lrREG=lrREG, lossREG=lossREG,
                beta1=beta1, beta2=beta2)
Reg.train(tr_in = DM.data_input_train[rndtr],
        tr_lb = DM.data_label_train[rndtr],
        ts_in = DM.data_input_test,
        ts_sv = DM.data_severity_test)

## Test model
In order to test the model, only feature extractor and severity regressor network block are required, and the script "regressor_only_for_test" only include those blocks.

PerformanceEvaluator class returns some value of metrics for performance evaluation, such as;
- accuracy plot
- BA plot
- roc plot
- accuracy, auc, sensitivity, specificity

In [ ]:
from lib.regressor_only_for_test import Regressor
from lib.performance_evaluator import PerformanceEvaluator

Reg = Regressor(device=device)
test_input, test_label, test_prediction, test_regession_loss = Reg.test(
    ts_in = DM.data_input_test, 
    ts_sv = DM.data_severity_test, 
    model_dir = "{}/{}".format(model_dir, model_name),
    epoch = epoch
)

PM = PerformanceEvaluator(
    test_label=test_label, 
    test_prediction=test_prediction, 
    test_regession_loss=test_regession_loss
)

# Measure accuracy of regression result.
PM.measures_accuracy(save_path="{}/accuracy_{}.png".format(result_dir, model_name))

# BA Plot
PM.draws_BA_plot(save_path="{}/BAplot_{}.png".format(result_dir, model_name))

# Performnace
PM.evaluates_performance(
    result_dir=result_dir,
    model_name=model_name,
    regressor_type = 'dl'
)

## Train & Test ABI
To compare the performance of DL, ABI would be used.

ABITester class would return the trained result of ABI.

In [ ]:
from lib.abi_tester import ABITester

AT = ABITester()
test_input, test_label, test_prediction, test_regession_loss
test_label, test_prediction, test_regession_loss = AT.test(
    tr_in = DM.data_input_train[rndtr],
    tr_sv = DM.data_severity_train[rndtr],
    ts_in = DM.data_input_test,
    ts_sv = DM.data_severity_test
)


PM = PerformanceEvaluator(
    test_label=test_label, 
    test_prediction=test_prediction, 
    test_regession_loss=test_regession_loss
)

# Measure accuracy of regression result.
PM.measures_accuracy(save_path="{}/accuracy_{}_abi.png".format(result_dir, model_name))

# BA Plot
PM.draws_BA_plot(save_path="{}/BAplot_{}.png_abi".format(result_dir, model_name))

# Performnace
PM.evaluates_performance(
    result_dir=result_dir,
    model_name=model_name,
    regressor_type = 'abi'
)